<a href="https://colab.research.google.com/github/HoangLong1907/DeepLearning-CS431/blob/main/Baitap/word2vec_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Để chạy được tutorial này, chúng ta cần phải cài đặt thư viện GenSim.
Để cài bằng Anaconda ta sử dụng lệnh sau:

$ conda install -c anaconda gensim

Sau đó import các thư viện cần thiết như sau:

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from gensim.models import KeyedVectors
import numpy as np

# Load model
Trước hết, ta cần tải model về theo đường dẫn sau:https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip

Chi tiết có thể tham khảo trên trang FastText https://fasttext.cc/docs/en/english-vectors.html. Đối với các ngôn ngữ khác, trong đó có tiếng Việt ta có thể tải về từ đây:
https://fasttext.cc/docs/en/crawl-vectors.html

Thời gian load mô hình đã huấn luyện này có thể mất từ 5-10 phút )-:


In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/DeepLearning/wiki-news-300d-1M.vec', binary=False)

# Word embeding: biểu diễn một từ dưới dạng một vector

In [ ]:
woman_vec = word_vectors['woman']
print('Feature representation of `woman` is:', woman_vec)
print('Size of feature representation: ', woman_vec.shape)

Feature representation of `woman` is: [ 7.510e-02 -5.910e-02 -1.020e-02 -4.900e-03 -2.230e-02  1.397e-01
  1.970e-02  1.058e-01 -6.740e-02  1.333e-01  8.540e-02  6.220e-02
  1.250e-01  8.060e-02  3.700e-03  2.580e-02  7.560e-02  8.550e-02
 -2.720e-02 -3.470e-02 -1.167e-01 -1.000e-04 -1.724e-01 -1.090e-02
  1.630e-02 -1.140e-02 -3.550e-02  1.869e-01  4.630e-02 -2.070e-02
  6.830e-02  3.230e-02  6.040e-02  1.790e-02 -5.800e-02 -3.860e-02
  1.900e-02  1.340e-01  1.246e-01  3.190e-02  1.430e-01 -3.370e-02
  6.510e-02 -1.368e-01 -6.050e-02 -5.230e-02  3.340e-02  6.600e-02
 -8.400e-03 -7.990e-02  1.318e-01 -6.720e-02 -6.142e-01  4.300e-03
  1.470e-02 -6.630e-02  3.920e-02  4.680e-02  9.020e-02 -2.040e-02
 -5.600e-03 -3.260e-02  5.350e-02 -8.250e-02 -5.250e-02 -2.490e-02
  1.021e-01  1.045e-01  1.810e-02 -5.670e-02 -6.390e-02 -1.349e-01
 -9.700e-03  1.113e-01 -1.544e-01  8.390e-02  1.040e-02 -5.720e-02
  4.600e-03  1.834e-01  1.060e-02  1.630e-01  8.160e-02 -2.479e-01
  9.800e-03  2.720e-02 -

# Tính độ tương đồng giữa hai từ
Mức độ tương đồng ở đây là tổng thể từ loại, cách sử dụng từ và vai trò trong câu.
Do đó, đừng bất ngờ khi độ tương đồng của 'woman' và 'man' lại cao hơn 'woman' và 'girl'.
Vai trò, chức năng của 'woman' và 'man' trong câu thường có tính đối ngẫu gần giống nhau. Trong khi 'woman' và 'girl' thì không đối ngẫu, do 'girl' là một tập con của 'woman' nên trong cấu trúc câu từ 'girl' đôi khi không thể thay thế bằng 'woman' được.

In [ ]:
print('Similarity of `woman` and `man`:', word_vectors.similarity('woman', 'man'))
print('Similarity of `woman` and `man`:', word_vectors.similarity('woman', 'girl'))
print('Similarity of `boy` and `man`:', word_vectors.similarity('boy', 'girl'))

Similarity of `woman` and `man`: 0.81645226
Similarity of `woman` and `man`: 0.744082
Similarity of `boy` and `man`: 0.8618077


# Mối quan hệ về không gian giữa các từ

Câu hỏi: Nếu như 'king' là 'man' thì 'queen' là gì?

Trong không gian vector để biểu diễn các từ, vị trí tương đối giữa các cặp từ có mối quan hệ tương đồng về ngữ nghĩa thì sẽ có khoảng cách tương đối giống nhau. Hình sau minh hoạ mối quan hệ về mặt không gian (vector) giữa các từ. Ta dự đoán rằng các vector biểu diễn này sẽ có sự tương đồng trong không gian, nghĩa là để trả lời câu hỏi trên, ta phải giải phương trình thể hiện mối quan hệ trong không gian như sau: 

vector('king') - vector('queen') = vector('man') - vector(X)

Suy ra: vector(X) = vector('man') - vector('king') + vector('queen').

Như vậy ta sẽ xác định xem những từ nào gần với phép biến đổi trên:
![title](king_and_queen.png)

In [ ]:
sim_words = word_vectors.most_similar_cosmul(positive=['Vietnam', 'pho'], negative=['wolrd'])
print(sim_words[0][0])
sim_words

Vietnamese


[('Vietnamese', 1.0888371467590332),
 ('Pho', 1.077613353729248),
 ('Nguyen', 1.0310759544372559),
 ('Huynh', 1.0143067836761475),
 ('Hanoi', 1.0122424364089966),
 ('Saigon', 1.006687045097351),
 ('Dinh', 1.0029217004776),
 ('Vuong', 1.0000678300857544),
 ('Viet', 0.9949368834495544),
 ('Danang', 0.9907997250556946)]

Như vậy, về mặt không gian vector, ta có thể biểu diễn được mối quan hệ ngữ nghĩa của các từ một cách đáng ngạc nhiên. Quan hệ trên là: Người đứng đầu - Giới tính.

Hoàn toàn tương tự, ta có thể khai thác được các mối quan hệ ngữ nghĩa khác như: Đất nước - Thủ Đô, Đất nước - món ăn, Công ty - Sản phẩm, Đất nước - Tổng thống. Ví dụ dưới đây minh hoạ cho mối quan hệ ngữ nghĩa: Đất nước - Thủ đô.

In [ ]:
sim_words = word_vectors.most_similar_cosmul(positive=['Vietnam', 'Berlin'], negative=['Germany'])
print('If Berlin is the capital of Germany then', sim_words[0][0], 'is the capital of Vietnam')

If Berlin is the capital of Germany then Hanoi is the capital of Vietnam


# Thử nghiệm
Các bạn hãy thử trên các quan hệ khác như: Đất nước - món ăn, Công ty - Sản phẩm, Đất nước - Tổng thống, Bang - Thành phố.